In [25]:
# Импорт библиотек
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pymorphy3
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer(lang='ru')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akasa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akasa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\akasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [26]:
# Функции для очистки текста

def clean_text(text):
    text = re.sub(r'<[^>]*>', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'[^а-яА-ЯёЁ\s-]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'[\s-]+', ' ', text)
    return text.strip().lower()

def remove_stopwords(text, language='russian'):
    base_stopwords = set(stopwords.words(language))
    keep_words = {
        'не', 'ни', 'нет', 'без', 'никак', 'вовсе', 'отнюдь',  # Отрицания
        'очень', 'совсем', 'абсолютно', 'совершенно', 'крайне',  # Интенсификаторы
        'ли', 'ведь', 'либо', 'даже',  # Модальные частицы
        'хорошо', 'плохо', 'ужасно', 'прекрасно'  # Оценочные прилагательные
    }
    final_stopwords = base_stopwords - keep_words
    words = word_tokenize(text, language='russian')
    return ' '.join([w for w in words if w.lower() not in final_stopwords])

def lemmatize_text(text):
    words = word_tokenize(text, language='russian')
    lemmas = []
    for word in words:
        parsed = morph.parse(word)[0]
        lemmas.append(parsed.normal_form)
    return ' '.join(lemmas)

def preprocess_data(df):
    df[text_column] = df[text_column].str.replace(r'\b\d+\b', '')
    df[text_column] = df[text_column].apply(clean_text)
    df[text_column] = df[text_column].apply(correct_spelling)
    df[text_column] = df[text_column].apply(remove_stopwords)
    df[text_column] = df[text_column].apply(lemmatize_text)
    return df

In [27]:
# Функции для визуализации
def plot_class_distribution(df, label_column='sentiment'):
    sns.countplot(data=df, x=label_column)
    plt.title('Распределение классов')
    plt.show()

def plot_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

def plot_text_length_distribution(df):
    df['text_length'] = df[text_column].apply(len)
    sns.histplot(data=df, x='text_length', bins=50)
    plt.title('Распределение длины текста')
    plt.show()

def plot_text_length_boxplot(df):
    df['text_length'] = df[text_column].apply(len)
    sns.boxplot(data=df, x='text_length')
    plt.title('Распределение длины текста (Boxplot)')
    plt.show()

def plot_top_n_words(df, n=20):
    words = ' '.join(df[text_column]).split()
    word_counts = Counter(words)
    top_words = word_counts.most_common(n)
    sns.barplot(x=[count for word, count in top_words], y=[word for word, count in top_words])
    plt.title(f'Топ-{n} самых частых слов')
    plt.show()

def visualization_of_the_dataset(df):
    plot_class_distribution(df)
    plot_text_length_distribution(df)
    all_text = ' '.join(df[text_column])
    plot_wordcloud(all_text)
    plot_text_length_boxplot(df)
    plot_top_n_words(df, n=20)

In [28]:
# Загрузка данных
data = pd.read_csv('datasets.csv')
text_column = "text"
data[text_column] = data[text_column].str.slice(0, 1500)

data.head()

,Unnamed: 0,text,sentiment
0,43956,Развода на деньги нет\nНаблюдаюсь в Лайфклиник...,1
1,17755,Отель выбрали потому что рядом со стадионом. О...,0
2,20269,"Вылечили\nГноился с рождения глазик, в поликли...",1
3,16648,Хорошее расположение.С вокзала дошли пешком.Но...,0
4,27879,"Отличное месторасположение,прекрасный вид,особ...",1


In [ ]:
data_clean = preprocess_data(data.copy())

data_clean.head()

In [ ]:
visualization_of_the_dataset(data_clean)

In [ ]:
data_clean.to_csv('data_clean.csv', index=False)